In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_ds = pd.read_csv("/kaggle/input/freesound-audio-tagging/train.csv")
test_ds = pd.read_csv("/kaggle/input/freesound-audio-tagging/test_post_competition.csv")
sample_sub = pd.read_csv("/kaggle/input/freesound-audio-tagging/sample_submission.csv")

In [28]:
torch.cuda.set_per_process_memory_fraction(0.5, device=0)

In [3]:
train_ds.columns

Index(['fname', 'label', 'manually_verified'], dtype='object')

In [4]:
path = "/kaggle/input/freesound-audio-tagging/audio_"

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AutoTokenizer, HubertForSequenceClassification, AutoFeatureExtractor
import librosa
from tqdm import tqdm
from transformers import AutoProcessor

2026-02-19 02:45:51.160705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771469151.382362      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771469151.436914      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771469151.927928      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771469151.927970      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771469151.927973      55 computation_placer.cc:177] computation placer alr

In [6]:
def extract(file):
    audio , sample_rate = librosa.load(file, sr = 16000)
    return audio, sample_rate

In [7]:
train_data = []
sample_rate = 0
for idx, row in  tqdm(train_ds.iterrows()):
    file =os.path.join(path + "train", row["fname"])
    audio, sample_rate = extract(file)
    train_data.append(audio)
    sample_rate = sample_rate

9473it [02:44, 57.45it/s]


In [8]:
test_data=[]
sample_rate_test =0
for idx , row in tqdm(test_ds.iterrows()):
    file = os.path.join(path + "test", row["fname"])
    audio, sample_rate= extract(file)
    test_data.append(audio)
    sample_rate_test =sample_rate

9400it [02:20, 66.68it/s]


In [9]:
from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()
train_ds["label"] = encoder.fit_transform(train_ds["label"])

In [10]:
processor = AutoFeatureExtractor.from_pretrained(
    "facebook/hubert-base-ls960"
)

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [11]:
train_data[1]

array([-6.3894142e-04, -1.1385819e-03, -1.1121513e-03, ...,
        3.8902181e-06, -7.2033909e-05, -2.2096296e-05], dtype=float32)

In [12]:
class AudioDataset(Dataset):
    def __init__(self, ds, audio_data, is_test=False):
        self.audio = audio_data
        self.is_test = is_test

        if not is_test:
            self.labels = ds["label"].values

    def __len__(self):
        return len(self.audio)

    def __getitem__(self, idx):
        audio = self.audio[idx]

        if isinstance(audio, list):
            audio = np.array(audio)

        if len(audio.shape) == 2:
            audio = audio.squeeze()

        if not self.is_test:
            return audio.astype(np.float32), self.labels[idx]
        else:
            return audio.astype(np.float32)


In [13]:
def collate_fn(batch):
    audios, labels = zip(*batch)

    inputs = processor(
        list(audios),
        sampling_rate=16000,
        padding=True,
        return_tensors="pt",
        return_attention_mask=True  
    )

    inputs["labels"] = torch.tensor(labels, dtype=torch.long)

    return inputs


In [14]:
train_df = AudioDataset(train_ds,train_data, is_test =False)
test_df=AudioDataset(test_ds, test_data, is_test =False)

train_dl = DataLoader(
    train_df,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers =4,
    pin_memory =True
)

test_dl = DataLoader(
    test_df,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)


In [15]:
for batch in train_dl:
    input_values = batch["input_values"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    print(labels)
    break


tensor([29, 14,  1, 27, 31, 29, 28, 20])


In [16]:
import torch.nn as nn
from transformers import HubertForSequenceClassification

class Model(nn.Module):
    def __init__(self, target_size):
        super().__init__() 
        
        self.model = HubertForSequenceClassification.from_pretrained(
            "facebook/hubert-base-ls960",
            num_labels=target_size,
            ignore_mismatched_sizes=True
        )
        for param in self.model.hubert.parameters():
            param.requires_grad = False
            
    def forward(self, input_values, attention_mask=None):
        outputs = self.model(
            input_values=input_values,
            attention_mask=attention_mask
        )    
        return outputs.logits

In [17]:
target_size = train_ds["label"].nunique()
device =("cuda" if torch.cuda.is_available() else "cpu")
model = Model(target_size= target_size ).to(device)
print(model)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model(
  (model): HubertForSequenceClassification(
    (hubert): HubertModel(
      (feature_extractor): HubertFeatureEncoder(
        (conv_layers): ModuleList(
          (0): HubertGroupNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (activation): GELUActivation()
            (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
          )
          (1-4): 4 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (5-6): 2 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
        )
      )
      (feature_projection): HubertFeatureProjection(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (projection): Linear(in_features=512, out_features=768, bias=True)
   

In [27]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset, DataLoader
import torch.nn as nn

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_df)):

    print(f"\n======== Fold {fold+1} ========")

    train_sub = Subset(train_df, train_idx)
    val_sub = Subset(train_df, val_idx)
    scaler = torch.cuda.amp.GradScaler()

    train_dl_sub = DataLoader(
        train_sub,
        batch_size=8,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=4,
        pin_memory=True
    )

    val_dl = DataLoader(
        val_sub,
        batch_size=8,
        shuffle=False,
        collate_fn=collate_fn
    )

    model = Model(target_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=1e-4,
        weight_decay=1e-5
    )

    for epoch in range(10):

        model.train()
        train_loss = 0

        for batch in train_dl_sub:
            ids = batch["input_values"].to(device)
            mask = batch.get("attention_mask", None)
            if mask is not None:
                mask = mask.to(device)

            y = batch["labels"].to(device)

            optimizer.zero_grad()

            with torch.amp.autocast("cuda"):
                outputs = model(ids, mask)
                loss = criterion(outputs, y)
        
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss+=(loss)

        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in val_dl:
                ids = batch["input_values"].to(device)
                mask = batch.get("attention_mask", None)
                if mask is not None:
                    mask = mask.to(device)

                y = batch["labels"].to(device)

                outputs = model(ids, mask)
                loss = criterion(outputs, y)

                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        accuracy = correct / total

        print(f"Epoch {epoch+1}")
        print(f"Train Loss: {train_loss/len(train_dl_sub):.4f}")
        print(f"Val Loss: {val_loss/len(val_dl):.4f}")
        print(f"Val Accuracy: {accuracy:.4f}")



======== Fold 1 ========


/tmp/ipykernel_55/1777444041.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB. GPU 0 has a total capacity of 14.56 GiB of which 53.81 MiB is free. Including non-PyTorch memory, this process has 14.51 GiB memory in use. Of the allocated memory 14.24 GiB is allocated by PyTorch, and 128.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [29]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()